### Evaluating Landsat-8 and Sentinel-2 Nadir BRDF Adjusted Reflectance (NBAR) on South of Brazil through a Reproducible and Replicable environment

## 1. Resource configuration

In [ ]:
import os

**Configure data directory (with environment variable)**

In [ ]:
data_directory = os.environ.get("DATA_DIRECTORY")
data_directory

**Defining input files and directories for Sentinel-2/MSI steps**

In [ ]:
#
# Input and files references
#
sentinel2_input_dir = os.path.join(data_directory, "raw_data/Sentinel2Data")

sentinel2_sceneid_list = os.path.join(data_directory, "raw_data/ReferenceFiles_L8-S2/s2-sceneids.txt")

**Defining input files and directories for Landsat-8/MSI steps**

In [ ]:
#
# Input and files references
#
landsat8_input_dir = os.path.join(data_directory, "raw_data/Landsat8Data")

landsat8_sceneid_list = os.path.join(data_directory, "raw_data/ReferenceFiles_L8-S2/l8-sceneids.txt")

**Defining auxiliary data**

In [ ]:
#
# Extra: LADS Auxiliary files
#
lads_auxiliary_data = os.path.join(data_directory, "raw_data/LADS_AuxiliaryData/")

**Defining output directory**

In [ ]:
derived_data_dir = os.path.join(data_directory, "derived_data")

**Load scene identifiers (Landsat-8/OLI and Sentinel-2/MSI)**

In [ ]:
from research_processing import toolbox

In [ ]:
#
# Landsat-8/OLI Scenes Identifiers
#
landsat8_sceneids = toolbox.standardize_filename(open(landsat8_sceneid_list).readlines())

#
# Sentinel-2/MSI Scenes Identifiers
#
sentinel2_sceneids = toolbox.standardize_filename(open(sentinel2_sceneid_list).readlines())

**Defining output scenes**

In [ ]:
#
# Landsat-8/OLI Scenes
#
l8_outputs = os.path.join(derived_data_dir, "l8")

#
# Sentinel-2/MSI Scenes
#
s2_outputs = os.path.join(derived_data_dir, "s2")

#
# Validation results
#
validation_outputs = os.path.join(derived_data_dir, "validation")

## 2. Atmosphere Correction

In [ ]:
# ToDo: Text

### 2.1. Sentinel-2/MSI (Sen2Cor)

In [ ]:
from research_processing.surface_reflectance import sen2cor

**Prepare output directory**

In [ ]:
sen2cor_dir = toolbox.prepare_output_directory(s2_outputs, "s2_sen2cor_sr")

**Apply sen2cor**

In [ ]:
sen2cor(sentinel2_input_dir, sen2cor_dir, sentinel2_sceneids)

**Saving Sen2Cor scene list**

In [ ]:
sen2cor_sceneids = toolbox.filename(os.listdir(sen2cor_dir))

### 2.2. Sentinel-2/MSI (LaSRC)

In [ ]:
from research_processing.surface_reflectance import lasrc

**Prepare output directory**

In [ ]:
lasrc_dir = toolbox.prepare_output_directory(s2_outputs, "s2_lasrc_sr")

**Apply LaSRC**

In [ ]:
lasrc(sentinel2_input_dir, lasrc_dir, sentinel2_sceneids, lads_auxiliary_data)

**Saving LaSRC scene list**

In [ ]:
lasrc_sceneids = toolbox.filename(os.listdir(lasrc_dir))

## 3. NBAR

In [ ]:
# ToDo: Text

### 3.1. Sentinel-2/MSI (Sen2Cor)

In [ ]:
from research_processing.nbar import s2_sen2cor_nbar

**Prepare output directory**

In [ ]:
s2_sen2cor_nbar_dir = toolbox.prepare_output_directory(s2_outputs, "s2_sen2cor_nbar")

**Generate NBAR for sen2cor products**

In [ ]:
s2_sen2cor_nbar(sen2cor_dir, s2_sen2cor_nbar_dir, sen2cor_sceneids)

**Saving Sen2Cor NBAR scene list**

In [ ]:
s2_sen2cor_nbar_sceneids = toolbox.filename(os.listdir(s2_sen2cor_nbar_dir))

### 3.2. Sentinel-2/MSI (LaSRC)

In [ ]:
from research_processing.nbar import s2_lasrc_nbar

**Prepare output directory**

In [ ]:
s2_lasrc_nbar_dir = toolbox.prepare_output_directory(s2_outputs, "s2_lasrc_nbar")

**Generate NBAR for LaSRC products**

In [ ]:
s2_lasrc_nbar(lasrc_dir, s2_lasrc_nbar_dir, lasrc_sceneids)

**Saving LaSRC NBAR scene list**

In [ ]:
s2_lasrc_nbar_sceneids = toolbox.filename(os.listdir(s2_sen2cor_nbar_dir))

### 3.3. Landsat-8/OLI (Surface Reflectance product)

In [ ]:
from research_processing.nbar import lc8_nbar, lc8_generate_angles

**Prepare output directory**

In [ ]:
lc8_nbar_dir = toolbox.prepare_output_directory(l8_outputs, "lc8_nbar")

**Generate Landsat-8 Angles for NBAR calculation**

In [ ]:
scene_angles_lc8 = lc8_generate_angles(landsat8_input_dir, landsat8_sceneids)

**Generate NBAR for Landsat-8/OLI Surface Reflectance products**

In [ ]:
lc8_nbar(landsat8_input_dir, lc8_nbar_dir, scene_angles_lc8)

**Saving Landsat-8/OLI NBAR scene list**

In [ ]:
lc8_nbar_scene_ids = toolbox.filename(os.listdir(lc8_nbar_dir))

## 4. Validations

In [ ]:
# ToDo: Text

In [ ]:
from research_processing.validation import validation_funcs
from research_processing.validation import validation_routines

### 4.1. Landsat-8 Surface Reflectance

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_sr_l8_pairs = validation_funcs.search_pairs_l8(landsat8_sceneid_list)

**Prepare output directory**

In [ ]:
validation_sr_l8_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_l8")

**Defining bands**

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_sr_l8_bands = ["B2", "B3", "B4", "B5", "B6", "B7"]

**Validate the results**

In [ ]:
validation_sr_l8_result = validation_routines.validation_sr_l8(
    input_dir  = landsat8_input_dir,
    cloud_dir  = landsat8_input_dir,
    output_dir = validation_sr_l8_dir,
    pairs      = validation_sr_l8_pairs, 
    bands      = validation_sr_l8_bands
)

### 4.2. Landsat-8 NBAR

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_nbar_l8_pairs = validation_funcs.search_pairs_l8(landsat8_sceneid_list)

**Prepare output directory**

In [ ]:
validation_nbar_l8_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_l8")

**Defining bands**

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_bands = ["B2", "B3", "B4", "B5", "B6", "B7"]

**Validate the results**

In [ ]:
validation_nbar_l8_results = validation_routines.validation_nbar_l8(
    input_dir  = lc8_nbar_dir,
    cloud_dir  = landsat8_input_dir,
    output_dir = validation_nbar_l8_dir,
    pairs      = validation_nbar_l8_pairs,
    bands      = validation_nbar_l8_bands
)

### 4.3. Sentinel-2/MSI Surface Reflectance (Sen2Cor)

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_sr_s2_sen2cor_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

**Prepare output directory**

In [ ]:
validation_sr_s2_sen2cor_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_s2_sen2cor")

**Defining bands**

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02).

Bands (10m)

In [ ]:
validation_sr_s2_sen2cor_bands10m = ["B02", "B03", "B04", "B08"]

Bands (20m)

In [ ]:
validation_sr_s2_sen2cor_bands20m = ["B8A", "B11", "B12"]

**Validate the results**

In [ ]:
validation_routines.validation_sr_s2_sen2cor(
    input_dir  = sen2cor_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_sr_s2_sen2cor_dir,
    pairs      = validation_sr_s2_sen2cor_pairs,
    bands10m   = validation_sr_s2_sen2cor_bands10m,
    bands20m   = validation_sr_s2_sen2cor_bands20m
)

### 4.4. Sentinel-2/MSI NBAR (Sen2Cor)

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_nbar_s2_sen2cor_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

**Prepare output directory**

In [ ]:
validation_nbar_s2_sen2cor_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_s2_sen2cor")

**Defining bands**

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02).

Bands (10m)

In [ ]:
validation_nbar_s2_sen2cor_bands10m = ["B02", "B03", "B04", "B08"]

Bands (20m)

In [ ]:
validation_nbar_s2_sen2cor_bands20m = ["B8A", "B11", "B12"]

**Validate the results**

In [ ]:
validation_routines.validation_nbar_s2_sen2cor(
    input_dir  = s2_sen2cor_nbar_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_nbar_s2_sen2cor_dir,
    pairs      = validation_nbar_s2_sen2cor_pairs,
    bands10m   = validation_nbar_s2_sen2cor_bands10m,
    bands20m   = validation_nbar_s2_sen2cor_bands20m
)

### 4.5. Sentinel-2/MSI (LaSRC)

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_sr_s2_lasrc_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

**Prepare output directory**

In [ ]:
validation_sr_s2_lasrc_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_s2_lasrc")

**Defining bands**

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a)

In [ ]:
validation_sr_s2_lasrc_bands = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

**Validate the results**

In [ ]:
validation_routines.validation_sr_s2_lasrc(
    input_dir  = lasrc_dir,
    cloud_dir  = sen2cor_dir,
    output_dir = validation_sr_s2_lasrc_dir,
    pairs      = validation_sr_s2_lasrc_pairs,
    bands      = validation_sr_s2_lasrc_bands
)

### 4.6. Sentinel-2/MSI NBAR (LaSRC)

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_nbar_s2_lasrc_pairs = validation_funcs.search_pairs_s2(sentinel2_sceneid_list)

**Prepare output directory**

In [ ]:
validation_nbar_s2_lasrc_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_s2_lasrc")

**Defining bands**

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a)

In [ ]:
validation_nbar_s2_lasrc_bands = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

**Validate the results**

In [ ]:
validation_routines.validation_nbar_s2_lasrc(
    input_dir  = s2_lasrc_nbar_dir, 
    cloud_dir  = sen2cor_dir, 
    output_dir = validation_nbar_s2_lasrc_dir, 
    pairs      = validation_nbar_s2_lasrc_pairs,
    bands      = validation_nbar_s2_lasrc_bands
)

### 4.7. Compare Sen2Cor SR (Landsat-8 and Sentinel-2)

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_sr_l8_s2_sen2cor_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

**Prepare output directory**

In [ ]:
validation_sr_l8_s2_sen2cor_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_l8_s2_sen2cor")

**Defining bands**

Landsat-8/OLI

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_sr_l8_s2_sen2cor_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

> Name of the spectral bands (spectral resolution = 10m and 20m) that will be used in the validation. These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02, B8A)

In [ ]:
validation_sr_l8_s2_sen2cor_bands_s2 = ["B02", "B03", "B04", "B08", "B8A", "B11", "B12"]

**Validate the results**

In [ ]:
validation_routines.validation_sr_l8_s2_sen2cor(
    input_dir_l8 = landsat8_input_dir, 
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = sen2cor_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_sr_l8_s2_sen2cor_dir,
    pairs        = validation_sr_l8_s2_sen2cor_pairs,
    bands_l8     = validation_sr_l8_s2_sen2cor_bands_l8,
    bands_s2     = validation_sr_l8_s2_sen2cor_bands_s2
)

### 4.8. Compare LaSRC SR (Landsat-8 and Sentinel-2)

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_sr_l8_s2_lasrc_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

**Prepare output directory**

In [ ]:
validation_sr_l8_s2_lasrc_dir = toolbox.prepare_output_directory(validation_outputs, "validation_sr_l8_s2_lasrc")

**Defining bands**

Landsat-8/OLI

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_sr_l8_s2_lasrc_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a).

In [ ]:
validation_sr_l8_s2_lasrc_bands_s2 = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

**Validate the results**

In [ ]:
validation_routines.validation_sr_l8_s2_lasrc(
    input_dir_l8 = landsat8_input_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = lasrc_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_sr_l8_s2_lasrc_dir,
    pairs        = validation_sr_l8_s2_lasrc_pairs,
    bands_l8     = validation_sr_l8_s2_lasrc_bands_l8,
    bands_s2     = validation_sr_l8_s2_lasrc_bands_s2
)

### 4.9. Compare Sen2Cor NBAR (Landsat-8 and Sentinel-2)

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_nbar_l8_s2_sen2cor_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

**Prepare output directory**

In [ ]:
validation_nbar_l8_s2_sen2cor_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_l8_s2_sen2cor")

**Defining bands**

Landsat-8/OLI

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_s2_sen2cor_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

> Name of the spectral bands (spectral resolution = 10m and 20m) that will be used in the validation. These names should be equivalent to the standard ESA Sentinel-2 .SAFE file band naming pattern (e.g. B01, B02, B8A)

In [ ]:
validation_nbar_l8_s2_sen2cor_bands_s2 = ["B02", "B03", "B04", "B08", "B8A", "B11", "B12"]

**Validate the results**

In [ ]:
validation_routines.validation_nbar_l8_s2_sen2cor(
    input_dir_l8 = lc8_nbar_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = s2_sen2cor_nbar_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_nbar_l8_s2_sen2cor_dir,
    pairs        = validation_nbar_l8_s2_sen2cor_pairs,
    bands_l8     = validation_nbar_l8_s2_sen2cor_bands_l8,
    bands_s2     = validation_nbar_l8_s2_sen2cor_bands_s2
)

### 4.10. Compare LaSRC NBAR (Landsat-8 and Sentinel-2)

In [ ]:
# ToDo: Text

**Search pairs**

In [ ]:
validation_nbar_l8_s2_lasrc_pairs = validation_funcs.search_pairs_l8_s2(
    landsat8_sceneid_list, sentinel2_sceneid_list
)

**Prepare output directory**

In [ ]:
validation_nbar_l8_s2_lasrc_dir = toolbox.prepare_output_directory(validation_outputs, "validation_nbar_l8_s2_lasrc")

**Defining bands**

Landsat-8/OLI

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard USGS Landsat-8 file band naming pattern (e.g. B1, B2)

In [ ]:
validation_nbar_l8_s2_lasrc_bands_l8 = ["B2", "B3", "B4", "B5", "B5", "B6", "B7"]

Sentinel-2/MSI

> Name of the spectral bands that will be used in the validation. These names should be equivalent to the standard LaSRC file band naming pattern (e.g. sr_band2, sr_band3, sr_band8a).

In [ ]:
validation_nbar_l8_s2_lasrc_bands_s2 = ["sr_band2", "sr_band3", "sr_band4", "sr_band8", "sr_band8a", "sr_band11", "sr_band12"]

**Validate the results**

In [ ]:
validation_routines.validation_nbar_l8_s2_lasrc(
    input_dir_l8 = lc8_nbar_dir,
    cloud_dir_l8 = landsat8_input_dir,
    input_dir_s2 = s2_lasrc_nbar_dir,
    cloud_dir_s2 = sen2cor_dir,
    output_dir   = validation_nbar_l8_s2_lasrc_dir, 
    pairs        = validation_nbar_l8_s2_lasrc_pairs,
    bands_l8     = validation_nbar_l8_s2_lasrc_bands_l8,
    bands_s2     = validation_nbar_l8_s2_lasrc_bands_s2
)